# ECODE example

In [ ]:
import json
import matplotlib.pyplot as plt

import bluepyopt as bpopt
import bluepyopt.ephys as ephys

import model
import evaluator
import time
import neuron
import plotting

import sys
import shutil

from pathlib import Path
import os

%matplotlib notebook

In [ ]:
sys.path.append('../ecode/')

In [ ]:
from generate_ecode_rheobase import generate_ecode_protocols, compute_rheobase_for_model

In [ ]:
probe_type = "linear" # linear 
model_name = 'hallermann' # "hallerman"
model_folder = (Path(".") / f"{model_name}_model").absolute()

In [ ]:
# compile model-specific mechanisms
if not (model_folder / "x86_64").is_dir():
    curr_dir = Path(".").absolute()
    os.chdir(model_folder)
    print(os.getcwd())
    os.system("nrnivmodl mechanisms")
    os.chdir(curr_dir)
else:
    print(f"Mechanisms already compiled for {model_name}")
    compiled_folder = Path("./x86_64")
    if compiled_folder.is_dir():
        shutil.rmtree(compiled_folder)
    shutil.copytree(model_folder / "x86_64", compiled_folder)

In [ ]:
cell = model.create(model_name=model_name, release=True)

probe = model.define_electrode(probe_type=probe_type)
# probe = None

param_names = [param.name for param in cell.params.values() if not param.frozen]
if model_name == "hallermann":
    cvode_active = False
else:
    cvode_active = True
sim = ephys.simulators.LFPySimulator(cell, cvode_active=cvode_active, electrode=probe)

In [ ]:
rheobase, rheo_protocols, rheo_responses = compute_rheobase_for_model(cell, sim=sim, step_min=0.3)

In [ ]:
ecode_protocols = generate_ecode_protocols(rheobase_current=rheobase, record_extra=True)

In [ ]:
for p in ecode_protocols["IDthresh"].protocols:
    print(p.recordings)

In [ ]:
ecode_responses = {}
for protocol_name, protocol in ecode_protocols.items():
    print(f"Running protocol {protocol_name}")
    t_start = time.time()
    response = protocol.run(cell_model=cell, param_values={}, sim=sim)
    ecode_responses.update(response)
    print(f"Elapsed time {protocol_name}: {time.time() - t_start}")    

In [ ]:
ecode_responses.keys()

In [ ]:
plotting.plot_responses(ecode_responses, protocol_names=["firepattern", "PosCheops"])